In [8]:
import pandas as pd
import psycopg2
import csv
from datetime import datetime

In [2]:
df=pd.read_csv(r'C:\Users\HP\week 7\building_data_warehouse\data\cleaned_scraped_data.csv')

In [3]:
df.head(5)

,channel title,channel username,ID,Message,Date,Media Path
0,Doctors Ethiopia,DoctorsET,864,https://youtu.be/5dboem-8kma?si=ldlueecnfuljvd...,2023-12-18 17:04:02+00:00,images\photo_2023-12-18_17-04-02.jpg
1,Doctors Ethiopia,DoctorsET,863,ዶክተርስ ኢትዮጵያ በ አዲስ አቀራረብ በ ቴሌቪዥን ፕሮግራሙን ለመጀመር ከ...,2023-11-03 16:14:39+00:00,images\photo_2023-11-03_16-14-39.jpg
2,Doctors Ethiopia,DoctorsET,862,ሞት በስኳር \n\nለልጆቻችን የምናሲዘው ምሳቃ ሳናቀው እድሚያቸውን ይቀን...,2023-10-02 16:37:39+00:00,images\photo_2023-10-02_16-37-39.jpg
3,Doctors Ethiopia,DoctorsET,861,ከ hiv የተፈወሰ ሰው አጋጥሟችሁ ያቃል ? ፈውስ እና ህክምና ?\n\nሙ...,2023-09-16 07:54:32+00:00,images\photo_2023-09-16_07-54-32.jpg
4,Doctors Ethiopia,DoctorsET,860,በቅርብ ጊዜ በሃገራችን ላይ እየተስተዋለ ያለ የተመሳሳይ ፆታ ( homos...,2023-09-01 16:16:15+00:00,images\photo_2023-09-01_16-16-15.jpg


In [5]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="postgres",
    user="postgres",
    password="1234"
)
cur = conn.cursor()

In [10]:
# Path to your CSV file
df= r'C:\Users\HP\week 7\building_data_warehouse\data\cleaned_scraped_data.csv'

In [12]:
# Function to validate date
def is_valid_date(date_str):
    try:
        # Attempt to parse the date
        datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
        return True
    except (ValueError, TypeError):
        return False

# Database connection
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="postgres",
    user="postgres",
    password="1234"
)
cur = conn.cursor()

# Open the CSV file and insert the data
with open(df, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row

    for row in reader:
        # Check if there's a valid date
        if is_valid_date(row[3]):
            date_value = row[3]
        else:
            date_value = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Use current timestamp as default

        # Insert the data
        cur.execute(
            """
            INSERT INTO telegram_data (channel_title, channel_username, Message, Date, Media_Path)
            VALUES (%s, %s, %s, %s, %s)
            """,
            (row[0], row[1], row[2], date_value, row[4])
        )

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

#the data is loaded sucessfully to my pg as transformed and cleaned